In [1]:
import os
import tensorflow as tf
import shutil
import pandas as pd
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
current_working_directory = os.getcwd() 
# print output to the console
print(current_working_directory)

2025-06-01 08:25:26.386650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748766326.400866   81318 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748766326.405085   81318 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748766326.416824   81318 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748766326.416835   81318 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748766326.416837   81318 computation_placer.cc:177] computation placer alr

/home/swanjohi/dual-view/mammo


In [2]:
csv_path = '/home/swanjohi/dual-view/csv/meta.csv'
df_meta = pd.read_csv(csv_path)
dicom_data = pd.read_csv('/home/swanjohi/dual-view/csv/dicom_info.csv')

In [3]:
image_dir = '/home/swanjohi/dual-view/jpeg'
full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
cropped_images = dicom_data[dicom_data.SeriesDescription == 'cropped images'].image_path
roi_mask_images = dicom_data[dicom_data.SeriesDescription == 'ROI mask images'].image_path
 
full_mammogram_images = full_mammogram_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
cropped_images = cropped_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
roi_mask_images = roi_mask_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
full_mammogram_images.iloc[0]

'/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590.100.1.2.248386742010678582309005372213277814849/1-249.jpg'

In [4]:
full_mammogram_images = full_mammogram_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
roi_mask_images = roi_mask_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
full_mammogram_images.iloc[0]
cropped_images.iloc[0]

'/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590.100.1.2.129308726812851964007517874181459556304/1-172.jpg'

In [5]:
full_mammogram_dict = dict()
cropped_dict = dict()
roi_mask_dict = dict()
 
for dicom in full_mammogram_images:
    # print(dicom)
    key = dicom.split("/")[5]
    # print(key)
    full_mammogram_dict[key] = dicom
for dicom in cropped_images:
    key = dicom.split("/")[5]
    cropped_dict[key] = dicom
for dicom in roi_mask_images:
    key = dicom.split("/")[5]
    roi_mask_dict[key] = dicom

In [6]:
sys.getsizeof(full_mammogram_dict)
 
mass_train_data = pd.read_csv('/home/swanjohi/dual-view/csv/mass_case_description_train_set.csv')
mass_test_data = pd.read_csv('/home/swanjohi/dual-view/csv/mass_case_description_test_set.csv')
calc_train_data = pd.read_csv('/home/swanjohi/dual-view/csv/calc_case_description_train_set.csv')
calc_test_data = pd.read_csv('/home/swanjohi/dual-view/csv/calc_case_description_test_set.csv')

In [7]:
def filter_dataframe_by_base_directory(df):

    base_directory = 'CBIS-DDSM/jpeg'
    # base_directory = 'content/jpeg"
 
    # Check if all three columns start with the base directory

    mask = (

        df['image file path'].str.startswith(base_directory) &

        df['cropped image file path'].str.startswith(base_directory) &

        df['ROI mask file path'].str.startswith(base_directory)

    )
 
    # Keep only the rows where all three columns start with the base directory

    filtered_df = df[mask]
 
    return filtered_df

In [8]:
def fix_image_path_mass(dataset):
    for i, img in enumerate(dataset.values):
        img_name = img[11].split("/")[2]
        if img_name in full_mammogram_dict:
            dataset.iloc[i, 11] = full_mammogram_dict[img_name]
 
        img_name = img[12].split("/")[2]
        if img_name in cropped_dict:
            dataset.iloc[i, 12] = cropped_dict[img_name]
 
        img_name = img[13].split("/")[2]
        if img_name in roi_mask_dict:
            dataset.iloc[i, 13] = roi_mask_dict[img_name]
 
fix_image_path_mass(mass_train_data)
 
fix_image_path_mass(mass_test_data)

In [9]:
mass_train = mass_train_data.rename(columns={'left or right breast': 'left_or_right_breast',
                                           'image view': 'image_view',
                                           'abnormality id': 'abnormality_id',
                                           'abnormality type': 'abnormality_type',
                                           'mass shape': 'mass_shape',
                                           'mass margins': 'mass_margins',
                                           'image file path': 'image_file_path',
                                           'cropped image file path': 'cropped_image_file_path',
                                           'ROI mask file path': 'ROI_mask_file_path'})

mass_test = mass_test_data.rename(columns={'left or right breast': 'left_or_right_breast',
                                           'image view': 'image_view',
                                           'abnormality id': 'abnormality_id',
                                           'abnormality type': 'abnormality_type',
                                           'mass shape': 'mass_shape',
                                           'mass margins': 'mass_margins',
                                           'image file path': 'image_file_path',
                                           'cropped image file path': 'cropped_image_file_path',
                                           'ROI mask file path': 'ROI_mask_file_path'})

In [10]:
mass_train

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,mass_shape,mass_margins,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,P_02033,2,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,3,MALIGNANT,4,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
1314,P_02079,2,RIGHT,CC,1,mass,ROUND,SPICULATED,3,MALIGNANT,5,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
1315,P_02079,2,RIGHT,MLO,1,mass,ROUND,SPICULATED,3,MALIGNANT,5,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...
1316,P_02092,2,LEFT,CC,1,mass,IRREGULAR,SPICULATED,3,MALIGNANT,2,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...,/home/swanjohi/dual-view/jpeg/1.3.6.1.4.1.9590...


In [11]:
def count_jpg_files_in_directory(directory):
    total_jpg_files = 0
 
    # Walk through the directory and count .jpg files
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                total_jpg_files += 1
 
    return total_jpg_files

In [12]:
mass_train.pathology.unique()
 
benign=mass_train[mass_train.pathology=="BENIGN"]
 
benignwoclbk=mass_train[mass_train.pathology=="BENIGN_WITHOUT_CALLBACK"]
malignant=mass_train[mass_train.pathology=="MALIGNANT"]

In [13]:
benignimgs=benign.image_file_path
malignantimgs=malignant.image_file_path
benignwoclbkimgs=benignwoclbk.image_file_path
ben='./class/benign'
benwcb='./class/benignwithoutcallback'
mal='./class/malignant'
os.makedirs(ben,exist_ok=True)
os.makedirs(mal,exist_ok=True)

# for colab/kaggle 
# ben = 'content/class/benign'


In [14]:
for ben_path in benignimgs:
  img_filename1=os.path.basename(ben_path)
  destination_path1=os.path.join(ben,img_filename1)
  shutil.copy(ben_path,destination_path1)
 
for mal_path in malignantimgs:
  img_filename1=os.path.basename(mal_path)
  destination_path1=os.path.join(mal,img_filename1)
  shutil.copy(mal_path,destination_path1)

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import efficientnet  # Example: EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
# Define data directory
data_dir = './class'
# Define image dimensions and batch size
image_height = 512
image_width = 512
batch_size = 32

In [17]:
# Data preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# Generate data batches
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

Found 432 images belonging to 2 classes.
Found 106 images belonging to 2 classes.


In [18]:
target_size=(image_height, image_width)

In [19]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, BatchNormalization

from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16

from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout

In [22]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [24]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16281877204971393378
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 45613383680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10732292254876809503
physical_device_desc: "device: 0, name: NVIDIA L40S, pci bus id: 0000:30:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


I0000 00:00:1748766790.796237   81318 gpu_device.cc:2019] Created device /device:GPU:0 with 43500 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:30:00.0, compute capability: 8.9


In [25]:
tf.test.is_built_with_cuda()  # Should return True
tf.test.is_gpu_available()    # Should return True

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


I0000 00:00:1748766822.406627   81318 gpu_device.cc:2019] Created device /device:GPU:0 with 43500 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:30:00.0, compute capability: 8.9


True

In [20]:
base_model = VGG16(weights=None, include_top=False, input_shape=target_size + (3,))
 
# Freeze the convolutional layers

for layer in base_model.layers:

    layer.trainable = False
 
# Add custom classification layers

model = Sequential([

    base_model,

    Flatten(),

    Dense(512, activation='relu'),

    BatchNormalization(),

    Dropout(0.5),

    Dense(256, activation='relu'),

    BatchNormalization(),

    Dropout(0.5),

    Dense(1, activation='sigmoid')

])
 
# Compile the model

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
 
# Train the model

I0000 00:00:1748766346.668706   81318 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43500 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:30:00.0, compute capability: 8.9
2025-06-01 08:25:47.364353: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:40] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /opt/cuda
  /home/swanjohi/ptenv/lib/python3.12/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/swanjohi/ptenv/lib/python3.12/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  /home/swanjohi/ptenv/lib/python3.12/site-packages/tensorflow/python/platform/../../cuda
  /home/swanjohi/ptenv/lib/

In [21]:
history = model.fit(

    train_generator,

    epochs=10,

    steps_per_epoch=len(train_generator),

    validation_data=val_generator,

    validation_steps=len(val_generator)

)

/home/swanjohi/ptenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1748766355.903064   81411 service.cc:152] XLA service 0x7e31e0012590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748766355.903090   81411 service.cc:160]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2025-06-01 08:25:55.939944: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748766356.295449   81411 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-06-01 08:25:56.634255: W external/local_xla/xla/stream_executor/gpu/redzone_allocator_kernel_cuda.cc:135] NOT_FOUND: Couldn't find a suitable version of ptxas. The following locations were considered: bin/ptxas, /home/swanjohi/ptenv/bin/ptxas, /usr/local/sbin/ptxas, /usr/local/bin/ptxas, /usr/sbin/ptxas, /usr/bin/ptxas, /sbin/ptxas, /bin/ptxas, /usr/games/ptxas, /usr/local/games/ptxas, /snap/bin/ptxas, ipykernel_launcher.runfiles/cuda_n

UnavailableError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/swanjohi/ptenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/swanjohi/ptenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/swanjohi/ptenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/swanjohi/ptenv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/swanjohi/ptenv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue



In [ ]:
 
# Evaluate the model

loss, accuracy = model.evaluate(val_generator, steps=len(val_generator))

print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

In [ ]:
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn import metrics
 
# Predict probabilities for the validation data

y_pred_prob = model.predict(val_generator)
 
# Convert probabilities to binary predictions (0 or 1)

y_pred = np.round(y_pred_prob).flatten()
 
# Get true labels from the validation generator

y_true = val_generator.classes
confusion_matrix = metrics.confusion_matrix(y_true, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()
 
# Compute confusion matrix

conf_matrix = confusion_matrix(y_true, y_pred)
 
print("Confusion Matrix:")

print(conf_matrix)